In [45]:
import numpy as np
import pandas as pd

In [46]:
df = pd.read_csv('train.csv')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [48]:
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

In [49]:
X = df.drop(columns=['PassengerId','Survived','Name','Ticket','Cabin'])
y = df['Survived']

In [50]:
from sklearn.preprocessing import LabelEncoder

features = ['Sex','Embarked']

for col in features:
    lbl = LabelEncoder()
    X[col] = lbl.fit_transform(list(df[col].values))

X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2


In [51]:
X['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [52]:
X['SibSp'].value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [53]:
from sklearn.preprocessing import StandardScaler


In [54]:
num_features = ['Age','Fare']

for col in num_features:
    scaler = StandardScaler()
    X[col] = scaler.fit_transform(np.array(df[col].values).reshape(-1,1))

X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,-0.592481,1,0,-0.502445,2
1,1,0,0.638789,1,0,0.786845,0
2,3,0,-0.284663,0,0,-0.488854,2
3,1,0,0.407926,1,0,0.420730,2
4,3,1,0.407926,0,0,-0.486337,2


In [55]:
X = pd.get_dummies(X,columns=['Pclass','SibSp','Embarked'],drop_first=True)
X.head()

,Sex,Age,Parch,Fare,Pclass_2,Pclass_3,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_1,Embarked_2
0,1,-0.592481,0,-0.502445,0,1,1,0,0,0,0,0,0,1
1,0,0.638789,0,0.786845,0,0,1,0,0,0,0,0,0,0
2,0,-0.284663,0,-0.488854,0,1,0,0,0,0,0,0,0,1
3,0,0.407926,0,0.420730,0,0,1,0,0,0,0,0,0,1
4,1,0.407926,0,-0.486337,0,1,0,0,0,0,0,0,0,1


In [56]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [57]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)

In [58]:
svm = SVC().fit(X_test,y_test)
svm.score(X_test,y_test)

0.8565022421524664

In [60]:
best_score = 0

for C in [1,10,100,1000,10000]:
    svm = SVC(C=C).fit(X_train,y_train)
    score = svm.score(X_test,y_test)

    if score > best_score:
        best_score = score
        best_parameter = C

print(best_parameter)
print(best_score)


10
0.8251121076233184


In [64]:
X_train_,X_validation,y_train_,y_validation = train_test_split(X_train,y_train,random_state=0)

In [66]:
best_score = 0

for C in [1,10,100,1000,10000]:
    svm = SVC(C=C).fit(X_train_,y_train_)
    score = svm.score(X_validation,y_validation)

    if score > best_score:
        best_score = score
        best_parameter = C

svm_best = SVC(C = best_parameter).fit(X_train,y_train)
svm_best.score(X_test,y_test)

0.8026905829596412

In [68]:
from sklearn.model_selection import  GridSearchCV


In [70]:
param = {
    'C':[0.01,0.1,1,10,100]
}

grid_search = GridSearchCV(SVC(),param_grid=param,cv=5)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVC(), param_grid={'C': [0.01, 0.1, 1, 10, 100]})

In [72]:
grid_search.best_params_

{'C': 1}

In [73]:
grid_search.best_score_

0.8128829536527886

In [74]:
grid_search.score(X_test,y_test)

0.8026905829596412

In [75]:
from sklearn.model_selection import cross_val_score

In [77]:
scores = cross_val_score(GridSearchCV(SVC(),param_grid=param,cv=5),X,y,cv=5)
print(scores)

[0.80446927 0.81460674 0.79213483 0.78089888 0.84831461]


In [78]:
scores.mean()

0.8080848659845584